<a href="https://colab.research.google.com/github/e-paj/M2M_Tech/blob/main/Capstone_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data

### Data Description

* The dataset consists of a longitudinal MRI data of 150 subjects aged 60 to 96.

* Each subject was scanned at least once.

* Everyone is right-handed.

* 72 of the subjects were grouped as 'Nondemented' throughout the study.

* 64 of the subjects were grouped as 'Demented' at the time of their initial visits and remained so throughout the study, with 51 individuals having mild to moderate Alzheimer's disease.

* 14 subjects were grouped as 'Nondemented' at the time of their initial visit and were subsequently characterized as 'Demented' at a later visit.


### Column Descriptors

Subject ID: Subject Identification

MRI ID: MRI Exam Identification

Group: Class, either 'Demented', 'Nondemented', or 'Other'

Visit: Order of visit

MR Delay: MR Delay Time (Contrast)

M/F: Gender of patient

Hand: left-handed or right-handed

Age: range from 60-98

EDUC: Years of Education

SES: Socioeconomic Status

MMSE: Mini Mental State Examination

CDR: Clinical Dementia Rating

eTIV: Estimated Total Intracranial Volume

nWBV: Normalize Whole Brain Volume

ASF: Atlas Scaling Factor

In [3]:
df = pd.read_csv('oasis_longitudinal.csv')
df.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


## Exploratory Data Analysis

### Import Libraries

In [45]:
import pandas as pd
import numpy as np
import graphviz
import sklearn
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import balanced_accuracy_score, accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

sklearn.set_config(display='diagram')
output_notebook()

To ensure any randomness throughout the code will be consistent.

In [ ]:
np.random.seed(5)

In [6]:
df = df.loc[df['Visit']==1] # use first visit only because of the analysis

df = df.reset_index(drop=True)

df['M/F'] = df['M/F'].replace(['F','M'],[0,1])

df['Group'] = df['Group'].replace(['Converted'],['Demented'])

df['Group'] = df['Group'].replace(['Demented','Nondemented'],[1,0])

df = df.drop(['MRI ID','Visit','Hand'], axis=1)

In [7]:
df.head()

,Subject ID,Group,MR Delay,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,0,0,1,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0002,1,0,1,75,12,NaN,23.0,0.5,1678,0.736,1.046
2,OAS2_0004,0,0,0,88,18,3.0,28.0,0.0,1215,0.710,1.444
3,OAS2_0005,0,0,1,80,12,4.0,28.0,0.0,1689,0.712,1.039
4,OAS2_0007,1,0,1,71,16,NaN,28.0,0.5,1357,0.748,1.293


In [58]:
# bar displays

groups = df.groupby('Group')
group0 = groups.get_group(0)
group1 = groups.get_group(1)

N_gen = group0.groupby('M/F').size().tolist()
D_gen = group1.groupby('M/F').size().tolist()

genders = ['Male', 'Female']

group = ['Demented', 'Nondemented']

In [59]:
data = {'Demented': D_gen, 'Nondemented': N_gen, 'Gender':['Female','Male']}
data

{'Demented': [38, 40], 'Nondemented': [50, 22], 'Gender': ['Female', 'Male']}

In [62]:
p = figure(x_range=genders, height=500, width=500,title = 'Gender and Demented Rate',
           x_axis_label='Gender',y_axis_label='Number of patients')

p.vbar_stack(group, x= 'Gender', color=['blue', 'red'], width=0.8, source=data,
             legend_label=group)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)